## The imports

In [1]:
import zarr
import xarray as xr
import time
import dask
from intake import open_catalog


# The workers

In [2]:
from dask.distributed import Client
from dask_gateway import Gateway

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.scale(32)
cluster


In [3]:
c = Client(cluster)
c

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.7597effc5da4423682711982a50afd41 Dashboard: /services/dask-gateway/clusters/prod.7597effc5da4423682711982a50afd41/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


Workers= 32, Cores=64, Memory=274.88 GB


# The data

In [5]:
%%time 

import xarray as xr
import gcsfs

fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)

zmap = fs.get_mapper('pangeo-meom/autre-eNATL60-BLBT02-SSH-1h/eNATL60-BLBT02-SSH-1h')
ds = xr.open_zarr(zmap)


CPU times: user 1.39 s, sys: 224 ms, total: 1.62 s
Wall time: 8.32 s


In [6]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 92.3 ms, sys: 1.97 ms, total: 94.3 ms
Wall time: 90.7 ms


In [7]:
%time mean.load()

KilledWorker: ("('xarray-sossheig-aa82661ee96d990edeac471f6fbb0de6', 10413, 0, 0)", <Worker 'tls://10.37.210.3:34861', name: dask-worker-7597effc5da4423682711982a50afd41-xbwsk, memory: 0, processing: 329>)

In [8]:
cluster.close()

Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.8/ssl.py", line 1309, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1124)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 696, in _handle_events
    self._handle